In [11]:
%load_ext autoreload
%autoreload 2

#import model
#import training
import loaders

import sys
from demoparser.demofile import DemoFile
import matplotlib.pyplot as plt
import time
import numpy as np

#Allow graphs to open in a window instead of inline
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
g = loaders.Game(0)

Loading game 0
Round  0
Round  1
Round  2
Round  3
Round  4
Round  5
Round  6
Round  7
Round  8
Round  9
Round  10
Round  11
Round  12
Round  13
Round  14
Round  15
Round  16
Round  17
Round  18
Round  19
Round  20
Round  21
Round  22
Round  23
Round  24
Round  25
Finished loading game!


In [57]:
r = g.m_rounds[0]

#All player data for a round

print(players)

l = max([len(p.states) for p in r.players])
print(l)

ls = [max([len(p.states) for p in rx.players]) for rx in g.m_rounds]
print(ls)

1392
[1392, 1742, 1840, 1853, 2007, 2060, 1173, 1614, 1329, 1920, 1838, 1488, 1695, 1446, 1812, 930, 1118, 1585, 1335, 1878, 2125, 980, 1979, 1139, 717, 1666]


1563.8846153846155

In [63]:
import math

batch_size = 32
network_z, network_x, network_y = 10, 128, 128
blob_size = 5.0

def get_batches():
    for b_ix in range(l//batch_size):  # Ignore bits at end that don't make up a whole batch
        #print("Batch", b_ix)
        Y = np.zeros((batch_size, network_z, network_x, network_y))

        # Generate batch
        for f in range(batch_size):
            i = b_ix*batch_size + f

            # Draw blob for each player
            for p_ix, p in enumerate(r.players):
                (pos, vis, dead) = p.states[i]
                if not dead:
                    put_heatmap(Y[p_ix, p_ix], pos)
                    
        yield Y

g = get_batches()
next(g)

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [50]:
a = Y[0,0]
print(a.shape)
plt.imshow(Y[0,0])

(128, 128)


In [38]:

# Get a batch of frames.
# This only gives the network target output, we have to generate it's input ourselves.
def get_next_batch():
    Y = np.empty((batch_size, network_z, network_x, network_y))

    # Only bother with full batches
    if current_frame + batch_size > len(something):
        return None

    for i in range(batch_size):
        Y[i,:] = get_frame(current_frame)
        current_frame += 1
    return Y

# Generate frame i
def get_frame(i):
    f = np.zeros((network_z, network_x, network_y))

    for p_ix in range(network_z):
        if is_alive(p_ix, i):
            put_heatmap(f[p_ix], x, y)

    return np.empty()

# Put a blob in the required place of a 2d matrix
# Can do this way faster, but it's fine for now..
def put_heatmap(heatmap, center):
    center_x, center_y = center
    
    # Rescale ..
    x1, x2, y1, y2 = (-2600., 2100., -1200., 3200.)
    center_x = (center_x - x1) / (x2 - x1) * network_x
    center_y = (center_y - y1) / (y2 - y1) * network_y
    
    height, width = heatmap.shape

    th = 4.6052
    delta = math.sqrt(th * 2)

    # Vectorize
    sigma = blob_size
    x0 = int(max(0, center_x - delta * sigma))
    y0 = int(max(0, center_y - delta * sigma))

    x1 = int(min(width - 1, center_x + delta * sigma))
    y1 = int(min(height - 1, center_y + delta * sigma))

    exp_factor = 1 / 2.0 / sigma / sigma
    arr_heatmap = heatmap[y0:y1 + 1, x0:x1 + 1] # Not nessasary unless we have > 1 blob per input
    y_vec = (np.arange(y0, y1 + 1) - center_y) ** 2  # y1 included
    x_vec = (np.arange(x0, x1 + 1) - center_x) ** 2
    xv, yv = np.meshgrid(x_vec, y_vec)
    arr_sum = exp_factor * (xv + yv)
    arr_exp = np.exp(-arr_sum)
    arr_exp[arr_sum > th] = 0
    heatmap[y0:y1 + 1, x0:x1 + 1] = np.maximum(arr_heatmap, arr_exp)
